## Task

### 0. Regex-ing

In [3]:
fields = ["password", "date_of_birth"]
messages = ["name=egg;email=eggmin@eggsample.com;password=eggcellent;date_of_birth=12/12/1986;", "name=bob;email=bob@dylan.com;password=bobbycool;date_of_birth=03/04/1993;"]

In [ ]:
# split the message using ;
# create a dict of k:v
# hide the data and convert back

In [63]:
def to_dict(msg) -> dict:
    return_dict = {}
    for item in msg:
        try: 
            # [return_dict.update(item) for item in msg]
                item = item.split("=")
                return_dict[item[0]] = item[1]
        except Exception:
            pass
    return return_dict

In [64]:
msg = ['name=egg', 'email=eggmin@eggsample.com', 'password=eggcellent', 'date_of_birth=12/12/1986', '']

to_dict(msg)

{'name': 'egg',
 'email': 'eggmin@eggsample.com',
 'password': 'eggcellent',
 'date_of_birth': '12/12/1986'}

In [69]:
from typing import List
import re

def filter_datum(fields: List[str], redaction: str,
                 message: str, separator: str) -> str:
    # split msg into parts
    new_message = message.split(separator)
    new_message_dict = to_dict(new_message)
    # hide specified data
    # pattern = "=([a-z]+|([0-9]+\/[0-9]+\/[0-9]+))"
    for field in fields:
        pattern = new_message_dict[field]
        message = re.sub(pattern, redaction, message)
    # print(result)
    # concat msg back
    # new_messages = ";".join(new_message)
    return message

In [70]:

for message in messages:
    print(filter_datum(fields, 'xxx', message, ';'))
    # filter_datum(fields, 'xxx', message, ';')

name=egg;email=eggmin@eggsample.com;password=xxx;date_of_birth=xxx;
name=bob;email=bob@dylan.com;password=xxx;date_of_birth=xxx;


In [55]:

for message in messages:
    print(filter_datum(fields, 'xxx', message, ';'))
    # filter_datum(fields, 'xxx', message, ';')

name=egg;email=eggmin@eggsample.com;xxxeggcellent;xxx12/12/1986;
name=bob;email=bob@dylan.com;xxxbobbycool;xxx03/04/1993;


In [84]:
import re

msg = 'name=egg;email=eggmin@eggsample.com;password=eggcellent;date_of_birth=12/12/1986;'
keys = ['date_of_birth', 'password']
# s = 'Make the World a *Better Place*'
for key in keys:
    pattern = key+r'(.*?);'
    repl = "xxx"
    replacement = f"{key}={repl};"
    msg = re.sub(pattern, replacement, msg)

print(msg)

name=egg;email=eggmin@eggsample.com;password=xxx;date_of_birth=xxx;


### 1. Log formatter

In [107]:
import logging


class RedactingFormatter(logging.Formatter):
    """ Redacting Formatter class
    """
    REDACTION = "***"
    FORMAT = "[HOLBERTON] %(name)s %(levelname)s %(asctime)-15s: %(message)s"
    SEPARATOR = ";"

    def __init__(self, fields: List[str]):
        super(RedactingFormatter, self).__init__(self.FORMAT)
        self.fields = fields

    def format(self, record: logging.LogRecord) -> str:
        """filter incoming records using `filter_datum`"""
        # convert record logger obj to string with
        # logging.Formatter.format() method
        mesg = super(__class__, self).format(record)  # instance of inherited class
        
        # print(mesg)
        return filter_datum(self.fields, self.REDACTION, mesg, self.SEPARATOR)


In [108]:
message = "name=Bob;email=bob@dylan.com;ssn=000-123-0000;password=bobby2019;"
log_record = logging.LogRecord("my_logger", logging.INFO, None, None, message, None, None)
formatter = RedactingFormatter(fields=("email", "ssn", "password"))
print(formatter.format(log_record))
# print(log_record.getMessage())
# log_record.msg = "abc"
# print(log_record.msg)


[HOLBERTON] my_logger INFO 2023-05-25 17:37:58,044: name=Bob;email=***;ssn=***;password=***;


In [104]:
log_record.levelname

'INFO'

In [87]:
message = "name=Bob;email=bob@dylan.com;ssn=000-123-0000;password=bobby2019;"


In [90]:
fields=("email", "ssn", "password")
filter_datum(fields,"***", message, ";")

'name=Bob;email=***;ssn=***;password=***;'

In [89]:
fields

('email', 'ssn', 'password')

### 2. Create logger

In [113]:
def get_logger() -> logging.Logger:
    logger = logging.getLogger("user_data")
    logger.setLevel(logging.INFO)
    logger.addHandler(logging.StreamHandler(__name__))
    return logger

In [114]:
get_logger()

<Logger user_data (INFO)>

In [115]:
print(get_logger.__annotations__.get('return'))


<class 'logging.Logger'>
